In [63]:
import pandas as pd
from datetime import datetime,timedelta
pd.read_csv("static/2022Sched.csv")

,Date,Number,Day of Week,Away,Away League,Away Season Game,Home,Home League,Home Season Game,Day/Night,Postponement,Makeup
0,20220407,0,Thu,CIN,NL,1,ATL,NL,1,n,NaN,NaN
1,20220407,0,Thu,CLE,AL,1,KCA,AL,1,d,NaN,NaN
2,20220407,0,Thu,HOU,AL,1,ANA,AL,1,n,NaN,NaN
3,20220407,0,Thu,MIL,NL,1,CHN,NL,1,d,NaN,NaN
4,20220407,0,Thu,NYN,NL,1,WAS,NL,1,d,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2425,20221005,0,Wed,SFN,NL,162,SDN,NL,162,n,NaN,NaN
2426,20221005,0,Wed,SLN,NL,162,PIT,NL,162,n,NaN,NaN
2427,20221005,0,Wed,TBA,AL,162,BOS,AL,162,n,NaN,NaN
2428,20221005,0,Wed,TOR,AL,162,BAL,AL,162,n,NaN,NaN


In [55]:
all_games=pd.read_csv("static/2022Sched.csv")

all_games["Date"]=pd.to_datetime(all_games["Date"].astype(str)).dt.strftime('%Y-%m-%d')
today_games = all_games[all_games["Date"] == "2022-04-20"].reset_index()

today_games["Date"][0].split

,index,Date,Number,Day of Week,Away,Away League,Away Season Game,Home,Home League,Home Season Game,Day/Night,Postponement,Makeup
0,172,2022-04-20,0,Wed,ANA,AL,13,HOU,AL,12,n,NaN,NaN
1,173,2022-04-20,0,Wed,ARI,NL,12,WAS,NL,14,n,NaN,NaN
2,174,2022-04-20,0,Wed,ATL,NL,14,LAN,NL,12,d,NaN,NaN
3,175,2022-04-20,0,Wed,BAL,AL,12,OAK,AL,13,n,NaN,NaN
4,176,2022-04-20,0,Wed,CHA,AL,12,CLE,AL,12,n,NaN,NaN
5,177,2022-04-20,0,Wed,CIN,NL,13,SDN,NL,14,d,NaN,NaN
6,178,2022-04-20,0,Wed,MIN,AL,12,KCA,AL,12,n,NaN,NaN
7,179,2022-04-20,0,Wed,NYA,AL,12,DET,AL,12,n,NaN,NaN
8,180,2022-04-20,0,Wed,PHI,NL,13,COL,NL,12,d,NaN,NaN
9,181,2022-04-20,0,Wed,PIT,NL,13,MIL,NL,14,d,NaN,NaN


In [168]:
from numpy import random
def game(away_rating,home_rating):
    difference=away_rating-home_rating
    result=random.normal(difference,1,100000)
    return result



In [178]:
sum(game(0.6,0)>0)/100000*162

117.70110000000001

In [179]:
sum([random.normal(0,1,15).max() for i in range(100000)])/100000

1.734108760491673